In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

from functools import reduce
from itertools import combinations

from scipy import stats

# configure pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Input: 0 or 1

In [2]:
home = 0

In [3]:
if home:
    dpath = '/Users/cglab/projects/abcd/data/abcd5.1-rser/'
else:
    dpath = '/home/cglab/projects/abcd/data/abcd5.1-rser/'

In [4]:
rs = pd.read_csv(dpath + 'abcd5.1_rtmri_2b_epn_cort_subc_net_ders_thrt_demo1_qcr_urg_dep.csv')
rs.shape

(20026, 560)

In [5]:
rs['eventname'].value_counts()

eventname
baseline_year_1_arm_1       9953
2_year_follow_up_y_arm_1    7219
4_year_follow_up_y_arm_1    2854
Name: count, dtype: int64

In [6]:
# transform eventname to columns
# except not for ders and fcon and erq here because those  aren't from those waves, and just repeated at each wave, 
# so only leave themin the rs1 so theyre not duplicated in the wide df
cols_same = [c for c in rs if 'ders' in c or 'Fcon' in c]
# pull out year 4
rs4 = rs[rs['eventname']=='4_year_follow_up_y_arm_1'].copy()
# exclude ders and erq columns from renaming because theyre already in wide format
ders_erq = [c for c in rs4.columns if 'ders' in c or 'erq' in c]
# drop ders and erq columns from rs4 and rs2, so they don't duplicate when merged with rs1
rs4.drop(columns=ders_erq, inplace=True)
yr4_cols = [c + '9' for c in rs4.columns if c!='subID']
rs4_cols = [c for c in rs4.columns if c!='subID']
rs4.rename(columns=dict(zip(rs4_cols, yr4_cols)), inplace=True)

# pull out year 2
rs2 = rs[rs['eventname']=='2_year_follow_up_y_arm_1'].copy()
# drop ders and erq columns from rs4 and rs2, so they don't duplicate when merged with rs1
rs2.drop(columns=ders_erq, inplace=True)
yr2_cols = [c + '5' for c in rs2.columns if c!='subID']
rs2_cols = [c for c in rs2.columns if c!='subID' or 'ders' in c or 'erq' in c]
rs2.rename(columns=dict(zip(rs2_cols, yr2_cols)), inplace=True)

# pull out year 1
rs1 = rs[rs['eventname']=='baseline_year_1_arm_1'].copy()
# get vars already in wide format, then remerge after wide transform
prewide = rs1[ders_erq + ['subID']].copy()
# drop ders and erq columns from rs4 and rs2, so they don't duplicate when merged with rs1
rs1.drop(columns=ders_erq, inplace=True)
yr1_cols = [c + '1' for c in rs1.columns if c!='subID']
rs1_cols = [c for c in rs1.columns if c!='subID' or 'ders' in c or 'erq' in c]
rs1.rename(columns=dict(zip(rs1_cols, yr1_cols)), inplace=True)

In [7]:
rs1.shape, rs2.shape, rs4.shape

((9953, 404), (7219, 404), (2854, 404))

In [8]:
rs1.head()

,sa_ad1,sa_cgc1,sa_ca1,sa_dla1,sa_fo1,sa_n1,sa_rspltp1,sa_sa1,sa_smh1,sa_smm1,sa_vs1,dt_ad1,dt_cgc1,dt_ca1,dt_dt1,dt_dla1,dt_fo1,dt_n1,dt_rspltp1,dt_sa1,dt_smh1,dt_smm1,dt_vs1,smh_smh1,dla_dla1,vta_ad1,vta_cgc1,vta_ca1,vta_dt1,vta_dla1,vta_fo1,vta_n1,vta_rspltp1,vta_sa1,vta_smh1,vta_smm1,vta_vta1,vs_vta1,subID,eventname1,sa_aalh1,sa_aarh1,sa_aglh1,sa_agrh1,sa_bs1,sa_cdelh1,sa_cderh1,sa_crcxlh1,sa_crcxrh1,sa_hplh1,sa_hprh1,sa_pllh1,sa_plrh1,sa_ptlh1,sa_ptrh1,sa_thplh1,sa_thprh1,sa_vtdclh1,sa_vtdcrh1,df_aalh1,df_aarh1,df_aglh1,df_agrh1,df_bs1,df_cdelh1,df_cderh1,df_crcxlh1,df_crcxrh1,df_hplh1,df_hprh1,df_pllh1,df_plrh1,df_ptlh1,df_ptrh1,df_thplh1,df_thprh1,df_vtdclh1,df_vtdcrh1,vta_aalh1,vta_aarh1,vta_aglh1,vta_agrh1,vta_cdelh1,vta_cderh1,vta_crcxlh1,vta_crcxrh1,vta_hplh1,vta_hprh1,vta_pllh1,vta_plrh1,vta_ptlh1,vta_ptrh1,vta_thplh1,vta_thprh1,vta_vtdclh1,vta_vtdcrh1,dsa_aalh1,dsa_aarh1,dsa_aglh1,dsa_agrh1,...,pInfParR1,pSupParL1,pSupParR1,pInfTmpL1,pInfTmpR1,pMidTmpL1,pMidTmpR1,pSupTmpL1,pSupTmpR1,tfmri_nback_all_8461,tfmri_nback_all_8801,tfmri_nback_all_8471,tfmri_nback_all_8811,pParHipL1,pParHipR1,pEntornL1,pEntornR1,pPTriFrL1,pPTriFrR1,pcACCL1,pcACCR1,prACCL1,prACCR1,nihtbx_totalcomp_agecorrected1,upps_y_ss_positive_urgency1,upps_y_ss_positive_urgency_nm1,upps_y_ss_negative_urgency1,upps_y_ss_negative_urgency_nm1,upps_y_ss_lack_of_planning1,upps_y_ss_lack_of_planning_nm1,upps_y_ss_sensation_seeking1,upps_y_ss_sensation_seeking_nm1,upps6_y1,upps7_y1,upps11_y1,upps12_y1,upps15_y1,upps16_y1,upps17_y1,upps18_y1,upps19_y1,upps20_y1,upps21_y1,upps22_y1,upps23_y1,upps24_y1,upps27_y1,upps28_y1,upps35_y1,upps36_y1,upps37_y1,upps39_y1,ADI11,ADI21,ADI31,ADI41,SocMob1,neigh11,neigh21,neigh31,scanID1,Motrs1,Motnb1,imgincl_rsfmri_include1,imgincl_nback_include1,famID1,age1,SES1,latent_factor_ss_social1,latent_factor_ss_perinatal1,gender1,pedu1,pedu21,income1,race1,prpensit1,ppensity1,FconP11,FconP21,FconP31,FconP41,FconP51,FconP61,FconP71,FconP81,FconP91,Sppensity1,SFconP11,SFconP21,SFconP31,SFconP41,SFconP51,SFconP61,SFconP71,SFconP81,SFconP91,neigh131,neigh231,neigh331,agey1
0,0.006416,0.021945,0.041819,-0.043350,0.215066,0.023004,-0.158642,0.511476,-0.110895,0.082378,-0.150867,-0.116451,-0.241669,-0.002354,0.345007,-0.234391,0.008572,0.090937,0.033278,0.110278,-0.112629,-0.100976,-0.088894,0.314437,0.350544,0.030371,0.017104,-0.147819,0.104240,-0.101375,0.058050,0.044380,-0.181789,0.156427,-0.042587,-0.042176,0.233819,-0.155134,NDAR_INV003RTV85,baseline_year_1_arm_1,0.161526,0.231572,0.175114,-0.104169,-0.050657,0.118217,0.414630,0.069444,-0.113659,-0.274035,-0.062580,-0.082889,0.096238,-0.029626,-0.083667,0.247253,0.117783,-0.129543,0.017039,0.329565,0.194284,-0.083103,0.057186,0.157341,-0.141171,-0.026373,-0.010895,-0.027565,0.030185,0.036097,0.166799,-0.098116,0.120868,-0.008540,-0.082872,-0.096747,0.182562,0.031598,-0.121166,-0.095990,-0.040662,-0.252743,0.100628,0.074931,-0.169070,0.059251,0.048614,0.180044,-0.039624,0.141900,-0.123925,-0.074228,-0.036094,-0.022273,-0.118111,0.087160,0.153251,0.102388,-0.127195,-0.122714,...,0.393528,0.514748,0.438104,0.397081,0.139539,0.363086,0.062205,0.210478,0.075170,-0.036367,0.265115,0.398479,0.311354,0.488468,0.217733,0.241092,0.268887,0.435629,0.376915,0.363426,0.490581,0.419764,0.494206,99.0,7.0,0.0,10.0,0.0,11.0,0.0,7.0,0.0,3.0,3.0,3.0,1.0,2.0,2.0,3.0,4.0,4.0,1.0,1.0,2.0,3.0,2.0,1.0,3.0,2.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,0.0,0.150697,0.139388,1.0,1.0,8781.0,131.0,0.1155,-0.9315,-0.0350,2.0,13.0,13.0,8.0,1.0,466.092707,466.092707,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.641641,-0.445586,-1.007864,-0.383727,-1.036719,-0.697481,-0.373403,2.838522,-0.458691,1.02604,5.0,5.0,4.0,10.916667
1,0.071985,0.108686,0.150118,-0.012935,0.114247,-0.011047,-0.100242,0.275316,-0.026035,0.001515,-0.017955,-0.138693,-0.188682,0.010733,0.273000,-0.095125,-0.034682,0.057536,0.159570,0.021238,-0.078270,-0.179893,-0.136426,0.288158,0.279062,0.076818,0.035681,-0.198201,0.041022,-0.099087,0.009414,0.026

In [9]:
rs2.head()

,sa_ad5,sa_cgc5,sa_ca5,sa_dla5,sa_fo5,sa_n5,sa_rspltp5,sa_sa5,sa_smh5,sa_smm5,sa_vs5,dt_ad5,dt_cgc5,dt_ca5,dt_dt5,dt_dla5,dt_fo5,dt_n5,dt_rspltp5,dt_sa5,dt_smh5,dt_smm5,dt_vs5,smh_smh5,dla_dla5,vta_ad5,vta_cgc5,vta_ca5,vta_dt5,vta_dla5,vta_fo5,vta_n5,vta_rspltp5,vta_sa5,vta_smh5,vta_smm5,vta_vta5,vs_vta5,subID,eventname5,sa_aalh5,sa_aarh5,sa_aglh5,sa_agrh5,sa_bs5,sa_cdelh5,sa_cderh5,sa_crcxlh5,sa_crcxrh5,sa_hplh5,sa_hprh5,sa_pllh5,sa_plrh5,sa_ptlh5,sa_ptrh5,sa_thplh5,sa_thprh5,sa_vtdclh5,sa_vtdcrh5,df_aalh5,df_aarh5,df_aglh5,df_agrh5,df_bs5,df_cdelh5,df_cderh5,df_crcxlh5,df_crcxrh5,df_hplh5,df_hprh5,df_pllh5,df_plrh5,df_ptlh5,df_ptrh5,df_thplh5,df_thprh5,df_vtdclh5,df_vtdcrh5,vta_aalh5,vta_aarh5,vta_aglh5,vta_agrh5,vta_cdelh5,vta_cderh5,vta_crcxlh5,vta_crcxrh5,vta_hplh5,vta_hprh5,vta_pllh5,vta_plrh5,vta_ptlh5,vta_ptrh5,vta_thplh5,vta_thprh5,vta_vtdclh5,vta_vtdcrh5,dsa_aalh5,dsa_aarh5,dsa_aglh5,dsa_agrh5,...,pInfParR5,pSupParL5,pSupParR5,pInfTmpL5,pInfTmpR5,pMidTmpL5,pMidTmpR5,pSupTmpL5,pSupTmpR5,tfmri_nback_all_8465,tfmri_nback_all_8805,tfmri_nback_all_8475,tfmri_nback_all_8815,pParHipL5,pParHipR5,pEntornL5,pEntornR5,pPTriFrL5,pPTriFrR5,pcACCL5,pcACCR5,prACCL5,prACCR5,nihtbx_totalcomp_agecorrected5,upps_y_ss_positive_urgency5,upps_y_ss_positive_urgency_nm5,upps_y_ss_negative_urgency5,upps_y_ss_negative_urgency_nm5,upps_y_ss_lack_of_planning5,upps_y_ss_lack_of_planning_nm5,upps_y_ss_sensation_seeking5,upps_y_ss_sensation_seeking_nm5,upps6_y5,upps7_y5,upps11_y5,upps12_y5,upps15_y5,upps16_y5,upps17_y5,upps18_y5,upps19_y5,upps20_y5,upps21_y5,upps22_y5,upps23_y5,upps24_y5,upps27_y5,upps28_y5,upps35_y5,upps36_y5,upps37_y5,upps39_y5,ADI15,ADI25,ADI35,ADI45,SocMob5,neigh15,neigh25,neigh35,scanID5,Motrs5,Motnb5,imgincl_rsfmri_include5,imgincl_nback_include5,famID5,age5,SES5,latent_factor_ss_social5,latent_factor_ss_perinatal5,gender5,pedu5,pedu25,income5,race5,prpensit5,ppensity5,FconP15,FconP25,FconP35,FconP45,FconP55,FconP65,FconP75,FconP85,FconP95,Sppensity5,SFconP15,SFconP25,SFconP35,SFconP45,SFconP55,SFconP65,SFconP75,SFconP85,SFconP95,neigh135,neigh235,neigh335,agey5
3,0.008133,0.061756,0.146352,-0.067697,0.102507,0.029304,-0.035043,0.425516,-0.163747,-0.096174,-0.136485,-0.082260,-0.205420,0.011431,0.341095,-0.183209,0.036926,0.118655,0.108350,0.123696,-0.036660,-0.157651,-0.243460,0.355581,0.263056,0.019050,0.010900,-0.105446,0.104743,-0.016388,0.027246,0.028950,-0.134220,0.131247,-0.042582,-0.035790,0.244665,-0.098899,NDAR_INV00CY2MDM,2_year_follow_up_y_arm_1,0.087313,0.032769,0.106473,-0.089008,0.039110,-0.045963,0.054209,-0.026604,-0.132791,-0.070338,0.041616,0.151400,0.006462,0.014982,-0.005261,0.219250,0.105179,-0.158763,0.169363,-0.067979,-0.030135,0.124339,-0.079521,0.024488,-0.081768,-0.040118,-0.067640,0.104975,-0.005856,0.023446,0.159807,0.007011,-0.111618,-0.002608,-0.030167,0.033741,-0.040782,0.062618,0.118414,-0.079090,0.055170,-0.072774,-0.068552,0.026533,0.126157,-0.009346,-0.034947,0.036080,-0.132658,0.082119,-0.026109,-0.042371,-0.107791,0.090810,-0.110814,0.033652,0.129193,-0.167324,-0.123083,0.064037,...,-0.125695,-0.235522,-0.177759,-0.325612,-0.150350,-0.066558,-0.160121,-0.063022,-0.039751,-0.044459,-0.214503,0.011422,-0.008402,0.049554,-0.085211,0.014037,-0.141380,-0.277141,-0.108081,-0.252381,-0.126206,0.163922,0.235349,NaN,8.0,0.0,8.0,0.0,6.0,0.0,9.0,0.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,4.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,4.0,0.072063,0.051531,1.0,1.0,NaN,152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1433.061575,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.112649,2.244238,0.992198,2.606020,0.964582,-0.697481,2.678070,2.838522,2.180116,1.02604,2.0,3.0,2.0,12.666667
6,0.101773,0.198276,0.156116,0.034849,0.084903,-0.016483,-0.166768,0.300588,0.059855,0.011500,-0.136875,-0.084754,-0.161390,-0.005210,0.280541,-0.201751,0.007305,0.054928,0.132438,0.052057,-0.090129,-0.124637,-0.226043,0.317535,0.380515,0.006295,-0.054085,-0.095234,0.114997,-0.117245,-0.004571,0.043125,0.060409,

In [10]:
rs4.head()

,sa_ad9,sa_cgc9,sa_ca9,sa_dla9,sa_fo9,sa_n9,sa_rspltp9,sa_sa9,sa_smh9,sa_smm9,sa_vs9,dt_ad9,dt_cgc9,dt_ca9,dt_dt9,dt_dla9,dt_fo9,dt_n9,dt_rspltp9,dt_sa9,dt_smh9,dt_smm9,dt_vs9,smh_smh9,dla_dla9,vta_ad9,vta_cgc9,vta_ca9,vta_dt9,vta_dla9,vta_fo9,vta_n9,vta_rspltp9,vta_sa9,vta_smh9,vta_smm9,vta_vta9,vs_vta9,subID,eventname9,sa_aalh9,sa_aarh9,sa_aglh9,sa_agrh9,sa_bs9,sa_cdelh9,sa_cderh9,sa_crcxlh9,sa_crcxrh9,sa_hplh9,sa_hprh9,sa_pllh9,sa_plrh9,sa_ptlh9,sa_ptrh9,sa_thplh9,sa_thprh9,sa_vtdclh9,sa_vtdcrh9,df_aalh9,df_aarh9,df_aglh9,df_agrh9,df_bs9,df_cdelh9,df_cderh9,df_crcxlh9,df_crcxrh9,df_hplh9,df_hprh9,df_pllh9,df_plrh9,df_ptlh9,df_ptrh9,df_thplh9,df_thprh9,df_vtdclh9,df_vtdcrh9,vta_aalh9,vta_aarh9,vta_aglh9,vta_agrh9,vta_cdelh9,vta_cderh9,vta_crcxlh9,vta_crcxrh9,vta_hplh9,vta_hprh9,vta_pllh9,vta_plrh9,vta_ptlh9,vta_ptrh9,vta_thplh9,vta_thprh9,vta_vtdclh9,vta_vtdcrh9,dsa_aalh9,dsa_aarh9,dsa_aglh9,dsa_agrh9,...,pInfParR9,pSupParL9,pSupParR9,pInfTmpL9,pInfTmpR9,pMidTmpL9,pMidTmpR9,pSupTmpL9,pSupTmpR9,tfmri_nback_all_8469,tfmri_nback_all_8809,tfmri_nback_all_8479,tfmri_nback_all_8819,pParHipL9,pParHipR9,pEntornL9,pEntornR9,pPTriFrL9,pPTriFrR9,pcACCL9,pcACCR9,prACCL9,prACCR9,nihtbx_totalcomp_agecorrected9,upps_y_ss_positive_urgency9,upps_y_ss_positive_urgency_nm9,upps_y_ss_negative_urgency9,upps_y_ss_negative_urgency_nm9,upps_y_ss_lack_of_planning9,upps_y_ss_lack_of_planning_nm9,upps_y_ss_sensation_seeking9,upps_y_ss_sensation_seeking_nm9,upps6_y9,upps7_y9,upps11_y9,upps12_y9,upps15_y9,upps16_y9,upps17_y9,upps18_y9,upps19_y9,upps20_y9,upps21_y9,upps22_y9,upps23_y9,upps24_y9,upps27_y9,upps28_y9,upps35_y9,upps36_y9,upps37_y9,upps39_y9,ADI19,ADI29,ADI39,ADI49,SocMob9,neigh19,neigh29,neigh39,scanID9,Motrs9,Motnb9,imgincl_rsfmri_include9,imgincl_nback_include9,famID9,age9,SES9,latent_factor_ss_social9,latent_factor_ss_perinatal9,gender9,pedu9,pedu29,income9,race9,prpensit9,ppensity9,FconP19,FconP29,FconP39,FconP49,FconP59,FconP69,FconP79,FconP89,FconP99,Sppensity9,SFconP19,SFconP29,SFconP39,SFconP49,SFconP59,SFconP69,SFconP79,SFconP89,SFconP99,neigh139,neigh239,neigh339,agey9
4,-0.104915,0.036135,0.181590,-0.048567,0.108634,0.055125,0.004793,0.469498,-0.307576,-0.253199,-0.041986,-0.175459,-0.293702,0.006998,0.373274,-0.184930,0.070347,0.096532,0.129323,0.096294,-0.162647,-0.218493,-0.271758,0.512576,0.225699,0.015750,0.007769,-0.004514,0.082624,-0.071732,0.007879,-0.009275,-0.088975,0.167178,-0.074667,-0.027951,0.296603,-0.062141,NDAR_INV00CY2MDM,4_year_follow_up_y_arm_1,0.032736,-0.019156,0.037819,-0.118584,0.093603,0.066322,0.314207,0.013684,-0.069731,-0.061999,-0.066355,0.047219,-0.004290,-0.021502,-0.025897,0.112690,0.146561,-0.091351,0.275422,0.029007,-0.043423,-0.080678,-0.150767,0.104675,-0.143908,-0.093628,0.054654,0.078445,0.049679,0.122489,0.006649,0.057579,-0.063977,-0.062197,-0.010774,0.012086,0.057337,0.139407,0.348450,0.054157,0.268174,-0.052472,-0.207925,-0.049149,0.166470,-0.055129,-0.039031,0.080863,-0.133220,0.230681,0.097654,-0.039276,-0.233856,0.066582,-0.242761,0.111459,0.174765,-0.122517,-0.089018,0.044553,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,0.0,8.0,0.0,7.0,0.0,9.0,0.0,2.0,2.0,1.0,3.0,2.0,1.0,3.0,3.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,4.0,0.113104,NaN,1.0,0.0,NaN,181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1433.061575,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.112649,2.244238,0.992198,2.606020,0.964582,-0.697481,2.678070,2.838522,2.180116,1.02604,2.0,3.0,2.0,15.083333
7,0.060360,0.159221,0.170096,0.012288,0.013780,0.003630,0.000995,0.264412,0.003320,-0.048580,-0.059406,-0.006139,-0.090234,0.053486,0.191875,-0.150371,0.007370,0.029690,0.029399,-0.004072,-0.027389,0.014906,-0.097645,0.323328,0.371339,0.050765,-0.021390,-0.061686,0.070919,-0.085110,0.000426,0.046233,-0.095212,-0.005174,-0.003050,0.096556,0.179587,-0.086857,NDAR_INV00HEV6HB,4_year_follow_up_y_arm_1,0.050222,-0.072623,0.004994,0.004864,-0.022071,0.20

In [11]:
[c for c in rs4.columns if 'subID' in c]

['subID']

In [12]:
# clear rs
rs = None
# merge the year1 and year2 into WIDE format
rs = rs1.merge(rs2, on='subID', how='left')
print(rs.shape)
# merge the rs with year4 into WIDE format
rs = rs.merge(rs4, on='subID', how='left')
print(rs.shape)
# merge the rs with prewide
rs = rs.merge(prewide, on='subID', how='left')
print(rs.shape)
rs.head()

(9953, 807)
(9953, 1210)
(9953, 1366)


,sa_ad1,sa_cgc1,sa_ca1,sa_dla1,sa_fo1,sa_n1,sa_rspltp1,sa_sa1,sa_smh1,sa_smm1,sa_vs1,dt_ad1,dt_cgc1,dt_ca1,dt_dt1,dt_dla1,dt_fo1,dt_n1,dt_rspltp1,dt_sa1,dt_smh1,dt_smm1,dt_vs1,smh_smh1,dla_dla1,vta_ad1,vta_cgc1,vta_ca1,vta_dt1,vta_dla1,vta_fo1,vta_n1,vta_rspltp1,vta_sa1,vta_smh1,vta_smm1,vta_vta1,vs_vta1,subID,eventname1,sa_aalh1,sa_aarh1,sa_aglh1,sa_agrh1,sa_bs1,sa_cdelh1,sa_cderh1,sa_crcxlh1,sa_crcxrh1,sa_hplh1,sa_hprh1,sa_pllh1,sa_plrh1,sa_ptlh1,sa_ptrh1,sa_thplh1,sa_thprh1,sa_vtdclh1,sa_vtdcrh1,df_aalh1,df_aarh1,df_aglh1,df_agrh1,df_bs1,df_cdelh1,df_cderh1,df_crcxlh1,df_crcxrh1,df_hplh1,df_hprh1,df_pllh1,df_plrh1,df_ptlh1,df_ptrh1,df_thplh1,df_thprh1,df_vtdclh1,df_vtdcrh1,vta_aalh1,vta_aarh1,vta_aglh1,vta_agrh1,vta_cdelh1,vta_cderh1,vta_crcxlh1,vta_crcxrh1,vta_hplh1,vta_hprh1,vta_pllh1,vta_plrh1,vta_ptlh1,vta_ptrh1,vta_thplh1,vta_thprh1,vta_vtdclh1,vta_vtdcrh1,dsa_aalh1,dsa_aarh1,dsa_aglh1,dsa_agrh1,...,Sders_upset_guilty_p_7,Sders_upset_guilty_p_9,Sders_upset_irritation_p_7,Sders_upset_irritation_p_9,Sders_upset_long_time_7,Sders_upset_long_time_9,Sders_upset_lose_control_7,Sders_upset_lose_control_9,Sders_upset_out_control_7,Sders_upset_out_control_9,Sders_upset_time_p_7,Sders_upset_time_p_9,Sders_upset_weak_p_7,Sders_upset_weak_p_9,ders_attn_awareness_p_7,ders_attn_awareness_p_9,ders_aware_clar_score_7,ders_aware_clar_score_9,ders_awareness_score_3_7,ders_awareness_score_4_9,ders_clear_feelings_p_7,ders_clear_feelings_p_9,ders_emotion_overwhelm_p_7,ders_emotion_overwhelm_p_9,ders_feelings_attentive_p_7,ders_feelings_attentive_p_9,ders_feelings_care_p_7,ders_feelings_care_p_9,ders_feelings_know_p_7,ders_feelings_know_p_9,ders_goals_score_3_7,ders_goals_score_4_9,ders_impulse_score_3_7,ders_impulse_score_4_9,ders_nonaccept_score_3_7,ders_nonaccept_score_4_9,ders_total_score_3_7,ders_total_score_4_9,ders_upset_ack_p_7,ders_upset_ack_p_9,ders_upset_angry_p_7,ders_upset_angry_p_9,ders_upset_ashamed_p_7,ders_upset_ashamed_p_9,ders_upset_behavior_control_7,ders_upset_behavior_control_9,ders_upset_behavior_p_7,ders_upset_behavior_p_9,ders_upset_better_p_7,ders_upset_better_p_9,ders_upset_concentrate_p_7,ders_upset_concentrate_p_9,ders_upset_control_p_7,ders_upset_control_p_9,ders_upset_depressed_p_7,ders_upset_depressed_p_9,ders_upset_difficulty_p_7,ders_upset_difficulty_p_9,ders_upset_embarrassed_p_7,ders_upset_embarrassed_p_9,ders_upset_emotion_overwhelm_7,ders_upset_emotion_overwhelm_9,ders_upset_esteem_p_7,ders_upset_esteem_p_9,ders_upset_feel_better_7,ders_upset_feel_better_9,ders_upset_fixation_p_7,ders_upset_fixation_p_9,ders_upset_focus_p_7,ders_upset_focus_p_9,ders_upset_guilty_p_7,ders_upset_guilty_p_9,ders_upset_irritation_p_7,ders_upset_irritation_p_9,ders_upset_long_time_7,ders_upset_long_time_9,ders_upset_lose_control_7,ders_upset_lose_control_9,ders_upset_out_control_7,ders_upset_out_control_9,ders_upset_time_p_7,ders_upset_time_p_9,ders_upset_weak_p_7,ders_upset_weak_p_9,erq_feelings_control_7,erq_feelings_control_9,erq_feelings_happy_7,erq_feelings_happy_9,erq_feelings_hide_7,erq_feelings_hide_9,erq_feelings_less_7,erq_feelings_less_9,erq_feelings_self_7,erq_feelings_self_9,erq_feelings_think_7,erq_feelings_think_9,erq_reapp_3_7,erq_reapp_4_9,erq_supp_3_7,erq_supp_4_9
0,0.006416,0.021945,0.041819,-0.043350,0.215066,0.023004,-0.158642,0.511476,-0.110895,0.082378,-0.150867,-0.116451,-0.241669,-0.002354,0.345007,-0.234391,0.008572,0.090937,0.033278,0.110278,-0.112629,-0.100976,-0.088894,0.314437,0.350544,0.030371,0.017104,-0.147819,0.104240,-0.101375,0.058050,0.044380,-0.181789,0.156427,-0.042587,-0.042176,0.233819,-0.155134,NDAR_INV003RTV85,baseline_year_1_arm_1,0.161526,0.231572,0.175114,-0.104169,-0.050657,0.118217,0.414630,0.069444,-0.113659,-0.274035,-0.062580,-0.082889,0.096238,-0.029626,-0.083667,0.247253,0.117783,-0.129543,0.017039,0.329565,0.194284,-0.083103,0.057186,0.157341,-0.141171,-0.026373,-0.010895,-0.027565,0.030185,0.036097,0.166799,-0.098116,0.120868,-0.008540,-0.082872,-0.096747,0.182562,0.0315

In [13]:
rs['upps11_y1'].unique()

array([ 3.,  1.,  2.,  4., nan])

### Convert Gender dummy and relative family id to integer
* for model ingestion
* Drop nonbinary gender
* for now

In [14]:
rs['gender1'].value_counts()

gender1
1.0      5082
2.0      4851
999.0       6
4.0         4
6.0         3
3.0         2
777.0       2
5.0         1
Name: count, dtype: int64

In [15]:
rs['gender1'] =  rs['gender1'].astype(float)
# rs.dropna(subset='rel_family_id_yr1', inplace=True)
rs['famID1'] =  rs['famID1'].astype(float)
# rs['gender'].dtype, rs['famID'].dtype
# drop non-binary genders for current analysis
rs = rs[(rs['gender1']==1) | (rs['gender1']==2)]

### UPPS scale 
* for each subscale at least 3 or 4 questinos should be answered
* so NaN subscale items if 3 questions aren't answered

* The num missing columns don't contain the number of missing questions as expect,
* so i will approach differently in future,

In [16]:
# upps_nm = {
#     'upps_y_ss_positive_urgency_nm': ['upps35_y', 'upps36_y', 'upps37_y', 'upps39_y'],
#     'upps_y_ss_negative_urgency_nm': ['upps7_y', 'upps11_y', 'upps17_y', 'upps20_y'],
#     'upps_y_ss_lack_of_planning_nm': ['upps6_y', 'upps16_y', 'upps23_y', 'upps28_y'],
#     'upps_y_ss_sensation_seeking_nm': ['upps12_y', 'upps18_y', 'upps21_y', 'upps27_y']}

In [17]:
# rs.loc[rs['upps_y_ss_positive_urgency_nm1'] < 3, ['upps35_y', 'upps36_y', 'upps37_y', 'upps39_y']]

In [18]:
# rs['upps_y_ss_positive_urgency_nm1'].value_counts()

In [19]:
# for num_miss, items in upps_nm.items():
#     for wave in ['1', '5', '9']:
#         rs.loc[rs[num_miss + wave] < 3, items] = np.nan
#         print((rs[num_miss + wave] < 3).sum())

### Done previously :Convert non-answer flags (777, 999) to nan

In [20]:
# rs[(rs==777) | (rs==-777) | (rs==999) | (rs==-999)]

In [21]:
# rs[(rs==777) | (rs==-777) | (rs==999) | (rs==-999)] = np.nan

### Save WIDE dataframe

In [22]:
rs.to_csv(dpath + 'abcd5.1_rtmri_2b_epn_cort_subc_net_ders_thrt_demo1_qcr_urg_dep_wide.csv', index=False)

# END

In [21]:
rs.describe()

,sa_ad1,sa_cgc1,sa_ca1,sa_dla1,sa_fo1,sa_n1,sa_rspltp1,sa_sa1,sa_smh1,sa_smm1,sa_vs1,dt_ad1,dt_cgc1,dt_ca1,dt_dt1,dt_dla1,dt_fo1,dt_n1,dt_rspltp1,dt_sa1,dt_smh1,dt_smm1,dt_vs1,smh_smh1,dla_dla1,vta_ad1,vta_cgc1,vta_ca1,vta_dt1,vta_dla1,vta_fo1,vta_n1,vta_rspltp1,vta_sa1,vta_smh1,vta_smm1,vta_vta1,vs_vta1,sa_aalh1,sa_aarh1,sa_aglh1,sa_agrh1,sa_bs1,sa_cdelh1,sa_cderh1,sa_crcxlh1,sa_crcxrh1,sa_hplh1,sa_hprh1,sa_pllh1,sa_plrh1,sa_ptlh1,sa_ptrh1,sa_thplh1,sa_thprh1,sa_vtdclh1,sa_vtdcrh1,df_aalh1,df_aarh1,df_aglh1,df_agrh1,df_bs1,df_cdelh1,df_cderh1,df_crcxlh1,df_crcxrh1,df_hplh1,df_hprh1,df_pllh1,df_plrh1,df_ptlh1,df_ptrh1,df_thplh1,df_thprh1,df_vtdclh1,df_vtdcrh1,vta_aalh1,vta_aarh1,vta_aglh1,vta_agrh1,vta_cdelh1,vta_cderh1,vta_crcxlh1,vta_crcxrh1,vta_hplh1,vta_hprh1,vta_pllh1,vta_plrh1,vta_ptlh1,vta_ptrh1,vta_thplh1,vta_thprh1,vta_vtdclh1,vta_vtdcrh1,dsa_aalh1,dsa_aarh1,dsa_aglh1,dsa_agrh1,dsa_cdelh1,dsa_cderh1,...,Sders_upset_guilty_p_7,Sders_upset_guilty_p_9,Sders_upset_irritation_p_7,Sders_upset_irritation_p_9,Sders_upset_long_time_7,Sders_upset_long_time_9,Sders_upset_lose_control_7,Sders_upset_lose_control_9,Sders_upset_out_control_7,Sders_upset_out_control_9,Sders_upset_time_p_7,Sders_upset_time_p_9,Sders_upset_weak_p_7,Sders_upset_weak_p_9,ders_attn_awareness_p_7,ders_attn_awareness_p_9,ders_aware_clar_score_7,ders_aware_clar_score_9,ders_awareness_score_3_7,ders_awareness_score_4_9,ders_clear_feelings_p_7,ders_clear_feelings_p_9,ders_emotion_overwhelm_p_7,ders_emotion_overwhelm_p_9,ders_feelings_attentive_p_7,ders_feelings_attentive_p_9,ders_feelings_care_p_7,ders_feelings_care_p_9,ders_feelings_know_p_7,ders_feelings_know_p_9,ders_goals_score_3_7,ders_goals_score_4_9,ders_impulse_score_3_7,ders_impulse_score_4_9,ders_nonaccept_score_3_7,ders_nonaccept_score_4_9,ders_total_score_3_7,ders_total_score_4_9,ders_upset_ack_p_7,ders_upset_ack_p_9,ders_upset_angry_p_7,ders_upset_angry_p_9,ders_upset_ashamed_p_7,ders_upset_ashamed_p_9,ders_upset_behavior_control_7,ders_upset_behavior_control_9,ders_upset_behavior_p_7,ders_upset_behavior_p_9,ders_upset_better_p_7,ders_upset_better_p_9,ders_upset_concentrate_p_7,ders_upset_concentrate_p_9,ders_upset_control_p_7,ders_upset_control_p_9,ders_upset_depressed_p_7,ders_upset_depressed_p_9,ders_upset_difficulty_p_7,ders_upset_difficulty_p_9,ders_upset_embarrassed_p_7,ders_upset_embarrassed_p_9,ders_upset_emotion_overwhelm_7,ders_upset_emotion_overwhelm_9,ders_upset_esteem_p_7,ders_upset_esteem_p_9,ders_upset_feel_better_7,ders_upset_feel_better_9,ders_upset_fixation_p_7,ders_upset_fixation_p_9,ders_upset_focus_p_7,ders_upset_focus_p_9,ders_upset_guilty_p_7,ders_upset_guilty_p_9,ders_upset_irritation_p_7,ders_upset_irritation_p_9,ders_upset_long_time_7,ders_upset_long_time_9,ders_upset_lose_control_7,ders_upset_lose_control_9,ders_upset_out_control_7,ders_upset_out_control_9,ders_upset_time_p_7,ders_upset_time_p_9,ders_upset_weak_p_7,ders_upset_weak_p_9,erq_feelings_control_7,erq_feelings_control_9,erq_feelings_happy_7,erq_feelings_happy_9,erq_feelings_hide_7,erq_feelings_hide_9,erq_feelings_less_7,erq_feelings_less_9,erq_feelings_self_7,erq_feelings_self_9,erq_feelings_think_7,erq_feelings_think_9,erq_reapp_3_7,erq_reapp_4_9,erq_supp_3_7,erq_supp_4_9
count,9881.000000,9877.000000,9881.000000,9881.000000,9881.000000,9871.000000,9881.000000,9881.000000,9874.000000,9881.000000,9881.000000,9881.000000,9877.000000,9881.000000,9881.000000,9881.000000,9881.000000,9871.000000,9881.000000,9881.000000,9874.000000,9881.000000,9881.000000,9874.000000,9881.000000,9880.000000,9877.000000,9880.000000,9880.000000,9880.000000,9880.000000,9870.000000,9880.000000,9880.000000,9874.000000,9880.000000,9880.000000,9880.000000,9881.000000,9881.000000,9881.000000,9880.000000,9881.000000,9881.000000,9881.000000,9874.000000,9881.000000,9877.000000,9881.000000,9881.000000,9881.000000,9880.000000,9874.000000,9881.000000,9871.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9874.000000,9881.000000,98

In [19]:
[c for c in rs.columns if 'erq' in c]

['erq_feelings_control_7',
 'erq_feelings_control_9',
 'erq_feelings_happy_7',
 'erq_feelings_happy_9',
 'erq_feelings_hide_7',
 'erq_feelings_hide_9',
 'erq_feelings_less_7',
 'erq_feelings_less_9',
 'erq_feelings_self_7',
 'erq_feelings_self_9',
 'erq_feelings_think_7',
 'erq_feelings_think_9',
 'erq_reapp_3_7',
 'erq_reapp_4_9',
 'erq_supp_3_7',
 'erq_supp_4_9']

In [26]:
rs['neigh11'].unique()

array([ 5.,  4.,  3.,  2.,  1., nan])

In [21]:
[c for c in rs if 'ADI' in c]

['ADI11',
 'ADI21',
 'ADI31',
 'ADI41',
 'ADI15',
 'ADI25',
 'ADI35',
 'ADI45',
 'ADI19',
 'ADI29',
 'ADI39',
 'ADI49']

In [22]:
[c for c in rs if 'pedu' in c]

['pedu1', 'pedu21', 'pedu5', 'pedu25', 'pedu9', 'pedu29']

In [15]:
[c for c in rs if 'ders' in c and '7' in c]

['Sders_attn_awareness_p_7',
 'Sders_aware_clar_score_7',
 'Sders_awareness_score_3_7',
 'Sders_clear_feelings_p_7',
 'Sders_emotion_overwhelm_p_7',
 'Sders_feelings_attentive_p_7',
 'Sders_feelings_care_p_7',
 'Sders_feelings_know_p_7',
 'Sders_goals_score_3_7',
 'Sders_impulse_score_3_7',
 'Sders_nonaccept_score_3_7',
 'Sders_total_score_3_7',
 'Sders_upset_ack_p_7',
 'Sders_upset_angry_p_7',
 'Sders_upset_ashamed_p_7',
 'Sders_upset_behavior_control_7',
 'Sders_upset_behavior_p_7',
 'Sders_upset_better_p_7',
 'Sders_upset_concentrate_p_7',
 'Sders_upset_control_p_7',
 'Sders_upset_depressed_p_7',
 'Sders_upset_difficulty_p_7',
 'Sders_upset_embarrassed_p_7',
 'Sders_upset_emotion_overwhelm_7',
 'Sders_upset_esteem_p_7',
 'Sders_upset_feel_better_7',
 'Sders_upset_fixation_p_7',
 'Sders_upset_focus_p_7',
 'Sders_upset_guilty_p_7',
 'Sders_upset_irritation_p_7',
 'Sders_upset_long_time_7',
 'Sders_upset_lose_control_7',
 'Sders_upset_out_control_7',
 'Sders_upset_time_p_7',
 'Sders_up